In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import copy


In [2]:
import numpy as np

In [3]:
import argparse
import wandb
import yaml
import random
import numpy as np

from training import train, evaluate
from config import Config
from getters import *
from models import *

from utils import find_file


/home/franck.mamalet/.conda/envs/deel-pt1.10/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:


def make_model(name, **kwargs):
    Model = get_model(name)
    return Model(**kwargs)

In [5]:
expe_name = 'copy_hadamssm_2'
task2sweep = {'copy_hadamssm': 'test',
              'copy_bjorck': 'sweep_ucvrt4w4',
              'copy_projunn': 'sweep_a4d2sihj',
             'pmnist_bjorck': 'sweep_bgckd728',
             'smnist': 'sweep_a3af3t6v',
             'ptb_bjorck': 'sweep_g0tjzjy7',
}

short2fullname = {
    'copy_hadamssm_2': 'hadamSSM_copy',
    'copy_bjorck_5' : 'fresh-sweep-19_qeh5ezkj',
    'copy_bjorck_6' : 'rich-sweep-25_eihml4ya',
    'copy_bjorck_8' : 'woven-sweep-33_dfdq24vp',
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 'rose-sweep-22_e3ftelec',
    'copy_projunn_8' : 'resilient-sweep-32_fze6523n',
    'pmnist_bjorck_3' : 'rare-sweep-1_uyv83nl9',
    'pmnist_bjorck_4' : 'rare4-sweep-1_uyv83nl9',
    'pmnist_bjorck_5' : 'fresh-sweep-2_c9nwwvj4',
    'pmnist_bjorck_6' : 'skilled-sweep-3_1pi0p71d',
    'pmnist_bjorck_8' : 'denim-sweep-4_27qjq5oa',
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 'sleek-sweep-2_275j9jq7',
    'smnist_bjorck_5' : 'wandering-sweep-3_1tbbz1et',
    'smnist_bjorck_6' : 'distinctive-sweep-4_c0gyg62r',
    'smnist_bjorck_8' : 'rich-sweep-5_ypocgzgv',
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 'earnest-sweep-2_rb6cg3gh',
    'ptb_bjorck_5' : 'eternal-sweep-3_5vss2or6',
    'ptb_bjorck_6' : 'polished-sweep-4_qa3yj567',
    'ptb_bjorck_8' : 'eager-sweep-5_whv7y4xu',
}

In [6]:
def get_task_value(expe_name,task2value):
    value = None
    for kk in task2value.keys():
        if kk in expe_name:
            value = task2value[kk]
            break
    return value

In [7]:
path2yaml = {'copy': 'copy_task',
             'pmnist': 'permuted_mnist',
             'smnist': 'seq_mnist',
             'ptb': 'ptb',}

config_path = get_task_value(expe_name,path2yaml)
print(config_path)

sweep_name = get_task_value(expe_name,task2sweep)

copy_task


In [8]:
sweep_name

'test'

In [9]:
config_file_path = "conf/"+config_path+"/"+short2fullname[expe_name]+".yaml"
weight_file_path = "results/"+sweep_name+"_"+short2fullname[expe_name]+".pth"
result_file_path = "results/"+sweep_name+"_"+short2fullname[expe_name]+".txt"

In [10]:
config_file_path = "conf/"+config_path+"/"+"copy_task_hadamSSM_config_test.yaml"


In [11]:
device = 'cpu' #'cpu' #'cpu' #'cuda:0'

In [12]:
config_file_path


'conf/copy_task/copy_task_hadamSSM_config_test.yaml'

## Verification of model load an performance with FP activations

In [13]:

conf_file = find_file(config_file_path)
config = Config(conf_file=conf_file)

In [14]:
config.model['num_bits_feat'] = None 

In [15]:

model = make_model(**config.model).to(device)
print(model)


BinadamSSM(
  (activation): ReLU()
  (input_layer): QLinear(in_features=10, out_features=512, bias=False)
  (recurrent_layer): ParametrizedBinadamard(
    in_features=512, out_features=512, bias=True
    (parametrizations): ModuleDict(
      (weight): ParametrizationList(
        (0): binadamard_parametrization()
      )
    )
  )
  (output_layer): Linear(in_features=512, out_features=9, bias=True)
)


In [16]:
model.load_state_dict(torch.load(weight_file_path))

<All keys matched successfully>

In [17]:
print(config.dataset)

{'name': 'CopyMemory', 'sent_length': 10, 'seq_length': 1000, 'test_size': 2000, 'train_size': 512000, 'val_size': 2000, 'vocabulary': 10}


In [18]:
Dataset = get_dataset(config.dataset['name'])

dataset = Dataset(**config.dataset)


------------------------------------------------------------Loading CopyMemory------------------------------------------------------------
------------------------------------------------------------CopyMemory loaded------------------------------------------------------------


In [19]:
device

'cpu'

In [20]:

if dataset.test_ds is not None:
    test_batch_size = dataset.te_size // 10
    test_ds = dataset.test_ds
else:
    test_batch_size = dataset.va_size // 10
    test_ds = dataset.val_ds
stat_test = evaluate(test_ds, test_batch_size, model, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device)
    

In [21]:
print(stat_test)
#ref  {'test_loss': 9.112985571846366e-05}   #in cuda {'test_loss': 0.00011134522355860099} ?????
# open and print result_file_path
with open(result_file_path, 'r') as f:
    print(f.read())


{'test_loss': 9.904126585524864e-08}
test_loss: 9.904285747097674e-08



## Prepare PTQ of activations:
Compute max value of activation on training set

TO BE DONE ONCE ONLY => CELLS IN MARKDOWN

In [22]:
weights = torch.load(weight_file_path)
new_weights = {}
new_weights["input_layer.weight"] = model.input_layer.qweight  #NO_PARAM model.input_layer.weight
new_weights['recurrent_layer.weight'] = model.recurrent_layer.weight
if 'activation.b' in weights:
    new_weights['activation.b'] = weights['activation.b']
if 'recurrent_layer.bias' in weights:
    new_weights['recurrent_layer.bias'] = weights['recurrent_layer.bias']
#new_weights['output_layer.weight'] = weights['output_layer.weight']

for kk in weights.keys():
    if (kk not in new_weights) and ('input_layer' not in kk) and ('recurrent_layer' not in kk):
        new_weights[kk] = weights[kk]

In [23]:
print(list(torch.unique(new_weights["input_layer.weight"]).numpy()))
print(list(torch.unique(new_weights["recurrent_layer.weight"]).detach().numpy()))

[-5.5423894, -2.7711947, 0.0, 2.7711947]
[-0.25, 0.0, 0.25]


In [24]:
config_scaled = copy.deepcopy(config)

config_scaled.model['num_bits']  = None
config_scaled.model['num_bits_feat'] = None
config_scaled.model['name'] = 'QSSM'
#config_scaled.model['bias'] = False
if 'activation.b' in new_weights:
    config_scaled.model['activation'] = layers.modrelu(**config_scaled.model['activation_config'])
else:
    config_scaled.model['activation'] = torch.nn.ReLU()
config_scaled.model['num_bits_feat'] = 16

In [25]:
device_stat = 'cuda'
model_stat = make_model(**config_scaled.model).to(device_stat)
model_stat.load_state_dict(new_weights)

<All keys matched successfully>

In [26]:
model_stat

QSSM(
  (activation): ReLU()
  (input_layer): QLinear(in_features=10, out_features=512, bias=False)
  (recurrent_layer): QLinear(in_features=512, out_features=512, bias=True)
  (output_layer): Linear(in_features=512, out_features=9, bias=True)
  (quant_input): _Quantize_stats()
  (quant_feat): _Quantize_stats()
)

In [32]:
#compute max_val on training set
train_batch_size = dataset.tr_size // 1000
stat_test = evaluate(dataset.train_ds, train_batch_size, model_stat, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device_stat)


In [33]:
print("stats on training set")
print(stat_test)
print("input")
print(model_stat.quant_input.stat_max_absolute)
print("feat")
print(model_stat.quant_feat.stat_max_absolute)

#stats on training set
#input
#1.0
#feat
#19.80474853515625

stats on training set
{'test_loss': 2.2125034604414395e-07}
input
1.0
feat
170.156494140625


In [27]:
#stat_test = evaluate(test_ds, test_batch_size, model_stat, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device_stat)
#print(stat_test)

In [28]:
expe_name

'copy_hadamssm_2'

In [29]:

task2input_max_absolute = {'copy': 2.0,
             'pmnist': 1.0,
             'smnist': 1.0,
             'ptb': 2.0,}

short2feat_max_absolute  = {
    'copy_hadamssm_2': 170.156494140625, #21.454788208007812,
    'copy_bjorck_5' : 24.19596290588379,
    'copy_bjorck_6' : 42.483619689941406,
    'copy_bjorck_8' : 30.833498001098633, #65.3037109375,
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 1902.8502197265625,
    'copy_projunn_8' : 219.551513671875,
    'pmnist_bjorck_3' : 22.018991470336914,
    'pmnist_bjorck_4' : 26.795150756835938,
    'pmnist_bjorck_5' : 25.203166961669922,
    'pmnist_bjorck_6' : 25.342670440673828,
    'pmnist_bjorck_8' : 26.046796798706055,
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 17.66891860961914,
    'smnist_bjorck_5' : 29.19618034362793,
    'smnist_bjorck_6' : 30.9416561126709,
    'smnist_bjorck_8' : 30.775388717651367,
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 30.078453063964844,
    'ptb_bjorck_5' : 6.935123443603516,
    'ptb_bjorck_6' : 6.048137187957764,
    'ptb_bjorck_8' : 9.994182586669922,
}

task2prescale = {'copy': 1.0,
             'pmnist': 255.0/256.0,
             'smnist': 255.0/256.0,
             'ptb': 1.0,}

task2shiftinput = {'copy': 0,
             'pmnist': 8,
             'smnist': 8,
             'ptb':0,}

In [30]:

saved_quant_input_max_absolute = get_task_value(expe_name,task2input_max_absolute) #short2input_max_absolute[expe_name] #2. # 2.  ## set it twice to get 1 as max
saved_quant_feat_max_absolute = short2feat_max_absolute[expe_name] #19.80474853515625   #32 #

input_pre_scaling_factor = get_task_value(expe_name,task2prescale)
#input_pre_scaling_factor = 1.0 # to get true Int
#input_pre_scaling_factor = 255.0/256.0
shift_input = get_task_value(expe_name,task2shiftinput)
#shift_input = 0
#shift_input = 8


In [31]:
print(saved_quant_input_max_absolute)
print(saved_quant_feat_max_absolute)
print(input_pre_scaling_factor)
print(shift_input)

2.0
170.156494140625
1.0
0


## Get $\alpha_W$ and $\alpha_U$ values

TO BE DONE ONCE ONLY => CELLS IN MARKDOWN

In [32]:
model.recurrent_layer.parametrizations.weight

ParametrizationList(
  (0): binadamard_parametrization()
)

In [33]:
'''model.recurrent_layer.parametrizations.weight.original
aa = model.recurrent_layer.parametrizations.weight[0](model.recurrent_layer.parametrizations.weight.original)
model.recurrent_layer.parametrizations.weight[-2](aa)'''

'model.recurrent_layer.parametrizations.weight.original\naa = model.recurrent_layer.parametrizations.weight[0](model.recurrent_layer.parametrizations.weight.original)\nmodel.recurrent_layer.parametrizations.weight[-2](aa)'

In [34]:
def computeParametrization(params,originaW,until_step=2):
    for kk in range(until_step):
        originaW = params[kk](originaW)
    return originaW

In [35]:
#Get floating point weights
import torch.nn.utils.parametrize as parametrize

print(model.recurrent_layer.parametrizations.weight)
if len(model.recurrent_layer.parametrizations.weight) > 1: #1 is only Quantize, SpectralLinear has 3 paramtrizations
    Wfp = computeParametrization(model.recurrent_layer.parametrizations.weight,
                                 model.recurrent_layer.parametrizations.weight.original,
                                 until_step=2) #model.recurrent_layer.parametrizations.weight[-2](model.recurrent_layer.parametrizations.weight.original)
else:
    Wfp = model.recurrent_layer.weight
print("W",Wfp.min().detach().numpy(),Wfp.max().detach().numpy(),np.max(np.abs(Wfp.detach().numpy())))
if parametrize.is_parametrized(model.input_layer):
    print(model.input_layer.parametrizations.weight)
    if len(model.input_layer.parametrizations.weight) > 1:
        Ufp = computeParametrization(model.input_layer.parametrizations.weight,
                                    model.input_layer.parametrizations.weight.original,
                                    until_step=2) #model.input_layer.parametrizations.weight[-2](model.input_layer.parametrizations.weight.original)
    else:
        Ufp = model.input_layer.parametrizations.weight.original
else:
    Ufp = model.input_layer.qweight #QLinear implementation of Armand 
print("U",Ufp.min().detach().numpy(),Ufp.max().detach().numpy(),np.max(np.abs(Ufp.detach().numpy())))

ParametrizationList(
  (0): binadamard_parametrization()
)
W -0.25 0.25 0.25
U -5.5423894 2.7711947 5.5423894


In [36]:

short2alpha_w  = {
    'copy_hadamssm_2': 0.25,
    'copy_bjorck_5' : 0.26512054,
    'copy_bjorck_6' : 0.28184956,
    'copy_bjorck_8' : 0.26092893, #0.26838166,
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 0.9988483,
    'copy_projunn_8' : 0.9998472,
    'pmnist_bjorck_3' : 0.32128334,
    'pmnist_bjorck_4' : 0.3660844,
    'pmnist_bjorck_5' : 0.34444174,
    'pmnist_bjorck_6' : 0.38659847,
    'pmnist_bjorck_8' : 0.6006793,
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 0.43379927,
    'smnist_bjorck_5' : 0.36560023,
    'smnist_bjorck_6' : 0.40944064,
    'smnist_bjorck_8' : 0.40728217,
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 0.19523722,
    'ptb_bjorck_5' : 0.23500842,
    'ptb_bjorck_6' : 0.16609415,
    'ptb_bjorck_8' : 0.48273855,
}

short2alpha_u  = {
    'copy_hadamssm_2': 5.5423894,
    'copy_bjorck_5' : 2.2110918,
    'copy_bjorck_6' : 2.2872915,
    'copy_bjorck_8' : 1.733867, #1.9520738,
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 7.8092656,
    'copy_projunn_8' : 1.9943593,
    'pmnist_bjorck_3' : 16.959444,
    'pmnist_bjorck_4' : 12.96517,
    'pmnist_bjorck_5' : 8.487175,
    'pmnist_bjorck_6' : 10.693445,
    'pmnist_bjorck_8' : 7.431476,
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 6.031232,
    'smnist_bjorck_5' : 8.092017,
    'smnist_bjorck_6' : 5.021599,
    'smnist_bjorck_8' : 6.9251447,
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 15.941487,
    'ptb_bjorck_5' : 2.4994867,
    'ptb_bjorck_6' : 2.4208195,
    'ptb_bjorck_8' : 7.541183,
}

In [37]:
saved_alpha_u = short2alpha_u[expe_name] #np.max(np.abs([-0.6072238087654114, 0.607637882232666]))
saved_alpha_w = short2alpha_w[expe_name] #nnp.max(np.abs([-0.24883395433425903, 0.24531713128089905]))

In [38]:
print(saved_alpha_u,saved_alpha_w)

5.5423894 0.25


## Rounding functions

In [39]:
def round_values(vv, num_bits):
    vv = vv*2**num_bits
    assert torch.max(torch.abs(vv.round() - vv))<1e-4   # ?????
    vv = vv.round()
    vv = vv/2**num_bits
    return vv

def verify_round_max(vv, num_bits):
    eval_numbits = 0
    for ss in range(num_bits):
        vv = vv*2
        if torch.max(torch.abs(vv.round() - vv))<1e-5:
            eval_numbits = ss+1
            break
    assert eval_numbits > 0
    print(eval_numbits,"max",num_bits)
    return eval_numbits
    
  
def verify_and_round(vv, num_bits):  
    eval_numbits = verify_round_max(vv, num_bits)
    return round_values(vv, eval_numbits)


## Fully Quantized RNN

In [40]:
weights = torch.load(weight_file_path)

In [41]:
vv = (2**config.model['num_bits'])*model.recurrent_layer.weight/ saved_alpha_w

In [42]:
torch.max(torch.abs(vv.round()-vv))

tensor(0., grad_fn=<MaxBackward1>)

In [43]:
len(torch.unique(model.input_layer.qweight))

4

In [44]:
new_weights = {}
requant_input_weight = round_values(model.input_layer.qweight/ saved_alpha_u, config.model['num_bits'])#NO_PARAM model.input_layer.weight
new_weights["input_layer.weight"] = requant_input_weight #model.input_layer.weight/ (saved_quant_input_max_absolute*saved_alpha_u)
float_recurrent_weight = model.recurrent_layer.weight
requant_recurrent_weight = round_values(model.recurrent_layer.weight/ saved_alpha_w, config.model['num_bits'])
new_weights['recurrent_layer.weight'] = requant_recurrent_weight #model.recurrent_layer.weight / saved_alpha_w ## compensate alpha_w in hidden quantization
#new_weights['recurrent_layer.weight'] = model.recurrent_layer.weight  ## compensate alpha_w in hidden quantization
scaled_quant_feat_max_absolute  = saved_quant_feat_max_absolute/saved_alpha_u
print(scaled_quant_feat_max_absolute) #saved_quant_feat_max_absolute*saved_alpha_w/(saved_quant_input_max_absolute*saved_alpha_u))
print("scaled hidden quantiz",scaled_quant_feat_max_absolute*saved_alpha_w)  #4.055123978191649
#new_quant_feat_max_absolute  = scaled_quant_feat_max_absolute #8/saved_alpha_w #saved_alpha_w*saved_quant_feat_max_absolute/(saved_quant_input_max_absolute*saved_alpha_u)
if 'activation.b' in weights:
    new_weights['activation.b'] = weights['activation.b']/ saved_alpha_u
    print("activation.b max abs",torch.abs(new_weights['activation.b']).max().cpu().detach().numpy())
if 'recurrent_layer.bias' in weights:
    new_weights['recurrent_layer.bias'] = weights['recurrent_layer.bias']/ saved_alpha_u
    print("'recurrent_layer.bias' max abs",torch.abs(new_weights['recurrent_layer.bias']).max().cpu().detach().numpy())
new_weights['output_layer.weight'] = saved_alpha_u*weights['output_layer.weight']/input_pre_scaling_factor

for kk in weights.keys():
    if (kk not in new_weights) and ('input_layer' not in kk) and ('recurrent_layer' not in kk):
        new_weights[kk] = weights[kk]

30.7009273185722
scaled hidden quantiz 7.67523182964305
'recurrent_layer.bias' max abs 0.19227177


In [45]:
print(saved_quant_feat_max_absolute,saved_alpha_u,saved_quant_feat_max_absolute/saved_alpha_u)
print(scaled_quant_feat_max_absolute,saved_alpha_w,scaled_quant_feat_max_absolute*saved_alpha_w)

170.156494140625 5.5423894 30.7009273185722
30.7009273185722 0.25 7.67523182964305


In [46]:

short2scaled_feat_max_absolute  = {  # if possible power of two
    'copy_hadamssm_2': 8.0,
    'copy_bjorck_5' : 4.0,
    'copy_bjorck_6' : 4.0,  
    'copy_bjorck_8' : 4.0, #12.0, #4.417504669503896
    'copy_projunn_5' : 'sweet-sweep-16_fvzfdbx1',
    'copy_projunn_6' : 256.0,
    'copy_projunn_8' : 128.0,
    'pmnist_bjorck_3' : 1.0,    #0.20856624553910355
    'pmnist_bjorck_4' : 1.0,    #0.7565875871836489
    'pmnist_bjorck_5' : 1.0,
    'pmnist_bjorck_6' : 1.0,
    'pmnist_bjorck_8' : 2.0,
    'smnist_bjorck_3' : 'cerulean-sweep-1_1k08ik92',
    'smnist_bjorck_4' : 2.0,
    'smnist_bjorck_5' : 2.0,
    'smnist_bjorck_6' : 4.0,
    'smnist_bjorck_8' : 2.0,
    'ptb_bjorck_3' : 'decent-sweep-1_5tay03df',
    'ptb_bjorck_4' : 1.0,
    'ptb_bjorck_5' : 1.0,
    'ptb_bjorck_6' : 1.0,
    'ptb_bjorck_8' : 1.0,
}

In [47]:
new_quant_feat_max_absolute  = short2scaled_feat_max_absolute[expe_name]  #saved_alpha_w*saved_quant_feat_max_absolute/(2*saved_alpha_u)

new_bias_max_absolute = 1.0

### check rounding of weights 

In [48]:
iww = 2**(config.model['num_bits']-1)*2*new_weights["input_layer.weight"].unique()
print("input ",torch.norm(iww.round() - iww))

iww = 2**(config.model['num_bits']-1)*requant_recurrent_weight
#print(iww)
print("recurrent ",torch.norm(iww.round() - iww))

input  tensor(0.)
recurrent  tensor(0., grad_fn=<CopyBackwards>)


### Evaluate with several bitwidth for activation

In [49]:
print(config.model)

{'activation': ReLU(), 'activation_config': None, 'activation_final': None, 'activation_final_config': None, 'basic_block': 4, 'bias': True, 'bias_final': True, 'givens': 2, 'hidden_size': 512, 'input_size': 10, 'manytomany': True, 'multiparameters': False, 'name': 'BinadamSSM', 'num_bits': 2, 'output_size': 9, 'seed': 171, 'single_layer': True, 'num_bits_feat': None}


In [50]:
import copy
config_scaled = copy.deepcopy(config)

config_scaled.model['num_bits']  = None
config_scaled.model['num_bits_feat'] = None
config_scaled.model['name'] = 'QSSM'
#config_scaled.model['bias'] = False
if 'activation.b' in new_weights:
    config_scaled.model['activation'] = layers.modified4q_modrelu(**config_scaled.model['activation_config'])
else:
    config_scaled.model['activation'] = torch.nn.ReLU()

In [50]:
from extra_layers import _Quantize_stats

qb = None
models = []
for qb in [None,12]: #6,8,10,12]:
    config_scaled.model['num_bits_feat'] = qb #10 #12
    model_scaled = make_model(**config_scaled.model).to(device)
    if qb is None:   # No rescaling of h with qb=None
        new_weights['recurrent_layer.weight'] = float_recurrent_weight
        #model_scaled.load_state_dict(new_weights_normalphaU)
    else:
        new_weights['recurrent_layer.weight'] = requant_recurrent_weight
    model_scaled.load_state_dict(new_weights)
    if qb is not None:      
        model_scaled.quant_input.pre_scaling_factor = input_pre_scaling_factor     
        model_scaled.quant_input.max_absolute = saved_quant_input_max_absolute #saved_quant_input_max_absolute  ## set it twice to get 1 as max
        model_scaled.quant_feat.max_absolute = new_quant_feat_max_absolute       
        model_scaled.quant_feat.pre_scaling_factor = saved_alpha_w     
        #print(model_scaled.activation.b)
        #print(quant_bias(model_scaled.activation.b))
        if ('activation.b' in new_weights) or ('recurrent_layer.bias' in new_weights):
            #quantize biases
            quant_bias = _Quantize_stats(weight=None,num_bits= model_scaled.quant_feat.num_bits, max_absolute= new_bias_max_absolute)
            if 'activation.b' in new_weights:
                model_scaled.activation.b.data = quant_bias(model_scaled.activation.b) 
            if 'recurrent_layer.bias' in new_weights:
                model_scaled.recurrent_layer.bias.data = quant_bias(model_scaled.recurrent_layer.bias)
    models.append(model_scaled)

In [51]:
x,y = next(iter(test_ds))
x0 = x[None,:] #No batch_size [0:1]
x0.shape

torch.Size([1, 1000, 10])

In [61]:

model_float = models[0]
model_float.eval()
model_float(x0.to(device))

print("input")
#print(model_float.quant_input.stat_max_absolute)
print("feat")
print(model_float.quant_feat.stat_max_absolute)

input
feat


AttributeError: 'NoneType' object has no attribute 'stat_max_absolute'

In [52]:
torch.unique(model_float.input_layer.weight)

NameError: name 'model_float' is not defined

In [61]:
model_scaled.quant_feat.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 'num_bits': 12,
 'enforce_true_quantized': True,
 'delta_m': None,
 'max_absolute': 8.0,
 'stat_max_absolute': 1e-10,
 'pre_scaling_factor': 0.25}

In [71]:
h0 = hidden_scaled[-1][0,220:230]
h1 = h0*model_scaled.quant_feat.pre_scaling_factor
h2 = h1/model_scaled.quant_feat.max_absolute
h3 = h2*2**(model_scaled.quant_feat.num_bits-1)
h4 = h3.round()
h5 = h4/2**(model_scaled.quant_feat.num_bits-1)
h6 = h5*model_scaled.quant_feat.max_absolute
h7 = h6/model_scaled.quant_feat.pre_scaling_factor
print(h0,"\n",h1,"\n",h2,"\n",h3,"\n",h4,"\n",h5,"\n",h6,"\n",h7)
print(torch.abs(h0-h7))
print(model_scaled.quant_feat(hidden_scaled[-1][0,220:230])/model_scaled.quant_feat.pre_scaling_factor)

tensor([ 0.1191,  0.1860,  0.4380, -0.0894,  0.1797, -0.0747,  0.1880, -0.3149,
        -0.3154,  0.1699], grad_fn=<SliceBackward0>) 
 tensor([ 0.0298,  0.0465,  0.1095, -0.0223,  0.0449, -0.0187,  0.0470, -0.0787,
        -0.0789,  0.0425], grad_fn=<MulBackward0>) 
 tensor([ 0.0037,  0.0058,  0.0137, -0.0028,  0.0056, -0.0023,  0.0059, -0.0098,
        -0.0099,  0.0053], grad_fn=<DivBackward0>) 
 tensor([  7.6250,  11.9062,  28.0312,  -5.7188,  11.5000,  -4.7812,  12.0312,
        -20.1562, -20.1875,  10.8750], grad_fn=<MulBackward0>) 
 tensor([  8.,  12.,  28.,  -6.,  12.,  -5.,  12., -20., -20.,  11.],
       grad_fn=<RoundBackward0>) 
 tensor([ 0.0039,  0.0059,  0.0137, -0.0029,  0.0059, -0.0024,  0.0059, -0.0098,
        -0.0098,  0.0054], grad_fn=<DivBackward0>) 
 tensor([ 0.0312,  0.0469,  0.1094, -0.0234,  0.0469, -0.0195,  0.0469, -0.0781,
        -0.0781,  0.0430], grad_fn=<MulBackward0>) 
 tensor([ 0.1250,  0.1875,  0.4375, -0.0938,  0.1875, -0.0781,  0.1875, -0.3125,
      

In [81]:
import torch.nn.functional as F
model_float = models[0]
model_scaled = models[1]
hidden = []
hidden_scaled = []
hidden_scaled_input = []
hidden_scaled_quant = []
hidden_scaled_rec = []
for ii in range(1000):
    print("input",x0[0:1,ii])
    print("qinput",model_scaled.quant_input(x0[0:1,ii])/input_pre_scaling_factor)
    #print(model.input_layer(x0[0:1,ii]))
    #print(model_scaled.input_layer(x0[0:1,ii])*saved_quant_input_max_absolute*saved_alpha_u)
    print("diff U.x",torch.norm(model_float.input_layer(x0[0:1,ii])-model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))/input_pre_scaling_factor))
    
    if len(hidden)==0:
        hidden.append(model_float.input_layer(x0[0:1,ii]))
        hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii])))
        #print(hidden_scaled[0])
        #print(hidden_scaled[0]*2**(config.model['num_bits']))
        hidden_scaled_quant.append(torch.zeros_like(hidden_scaled[0])) # fake
        hidden_scaled_rec.append(torch.zeros_like(hidden_scaled[0])) # fake
    else:
        #print("hidden",hidden[-1][0,0:10])
        #print("hidden_rec",model.recurrent_layer(hidden[-1])[0,0:10])
        hidden.append(model_float.input_layer(x0[0:1,ii])+model_float.recurrent_layer(hidden[-1]))
        #print("hidden_scaled",hidden_scaled[-1][0,0:10])
        #print("hidden_scaled",hidden_scaled[-1][0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        hidden_scaled_quant.append(model_scaled.quant_feat(hidden_scaled[-1]))
        #print("hidden_scaled_quant",hidden_scaled_quant[-1][0,0:10])
        #print("hidden_scaled_quant",hidden_scaled_quant[-1][0,0:10]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor))
        print("diff_hidden_scaled",torch.norm(hidden[-2]-hidden_scaled[-1]/input_pre_scaling_factor))
        print("diff_hidden_quant",torch.norm(hidden[-2]-hidden_scaled_quant[-1]/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor)))
        diff_hidden_scaled = (hidden_scaled_quant[-1]/model_scaled.quant_feat.pre_scaling_factor-hidden_scaled[-1])*saved_alpha_u/input_pre_scaling_factor
        print("quant diff hidden",torch.norm(diff_hidden_scaled).detach().numpy(),torch.min(torch.abs(diff_hidden_scaled)).detach().numpy(),torch.mean(torch.abs(diff_hidden_scaled)).detach().numpy(),torch.max(torch.abs(diff_hidden_scaled)).detach().numpy(),diff_hidden_scaled.shape)
        #print("hidden_scaled_reccccc",model.recurrent_layer(hidden_scaled_quant[-1]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor))[0,0:10])
        #print("hidden_scaled_reccccc1",model.recurrent_layer(hidden_scaled_quant[-1]/model_scaled.quant_feat.pre_scaling_factor)[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        #print("hidden_scaled_rec",model_scaled.recurrent_layer(hidden_scaled_quant[-1])[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        #print("hidden_scaled_rec",model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))+model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1])))
        hidden_scaled_rec.append(model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))) 
    
    hidden_scaled_input.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii])))
    #print("hidden_scaled_rec",hidden_scaled_rec[-1]*saved_alpha_u/input_pre_scaling_factor)
    print("diff h_t",torch.norm(hidden[-1]-hidden_scaled[-1]/input_pre_scaling_factor))
    print("h_t, min,mean,max,norm",hidden[-1].min().detach().numpy(),hidden[-1].mean().detach().numpy(),hidden[-1].max().detach().numpy(),torch.norm(hidden[-1]).detach().numpy())
    #print("diff h_t",hidden[-1][0,0:10])
    #print("diff h_t",hidden_scaled[-1][0,0:10])
    #print("diff h_t",hidden_scaled[-1][0,0:10]*saved_alpha_u/input_pre_scaling_factor)
    #if ii>2:
    #    aaa
    if 'activation.b' in new_weights:
        print("diff h_t+bias",torch.norm(hidden[-1]+model.activation.b-(hidden_scaled[-1]+model_scaled.activation.b)*saved_alpha_u/input_pre_scaling_factor))
    #NO ACTIV IN SSM hidden[-1] = model.activation(hidden[-1])
    #NO ACTIV IN SSM hidden_scaled[-1] = model_scaled.activation(hidden_scaled[-1])
    print("activ",torch.norm(hidden[-1]-hidden_scaled[-1]/input_pre_scaling_factor),torch.norm(hidden[-1]-hidden_scaled[-1]/input_pre_scaling_factor)/torch.norm(hidden[-1]),torch.max(torch.abs(hidden[-1]-hidden_scaled[-1]/input_pre_scaling_factor)))
    #print("activ value",hidden[-1],hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor)
    #print("activ valueq",hidden_scaled[-1],model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor)
    print("activ quantif",torch.norm(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor/input_pre_scaling_factor))
    '''if torch.max(torch.abs(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor/input_pre_scaling_factor))>0.1:
        print(hidden[-1][0,290:300])
        print(hidden_scaled[-1][0,290:300]/input_pre_scaling_factor)
        print(model_scaled.quant_feat(hidden_scaled[-1])[0,290:300]/model_scaled.quant_feat.pre_scaling_factor/input_pre_scaling_factor)
        diff = hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor/input_pre_scaling_factor
        print(torch.argmax(torch.abs(diff)))
        print("diff",diff[0,290:300],diff.min().detach().numpy(),diff.mean().detach().numpy(),diff.max().detach().numpy(),torch.norm(diff).detach().numpy())
        break'''

input tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
qinput tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
diff U.x tensor(0., grad_fn=<CopyBackwards>)
diff h_t tensor(0., grad_fn=<CopyBackwards>)
h_t, min,mean,max,norm -0.5 -0.0048828125 0.5 2.7838821
activ tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<DivBackward0>) tensor(0., grad_fn=<MaxBackward1>)
activ quantif tensor(0., grad_fn=<CopyBackwards>)
input tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
qinput tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
diff U.x tensor(0., grad_fn=<CopyBackwards>)
diff_hidden_scaled tensor(0., grad_fn=<CopyBackwards>)
diff_hidden_quant tensor(0., grad_fn=<CopyBackwards>)
quant diff hidden 0.0 0.0 0.0 0.0 torch.Size([1, 512])
diff h_t tensor(0.0032, grad_fn=<CopyBackwards>)
h_t, min,mean,max,norm -0.7978067 -0.0029966324 0.92874837 4.1673536
activ tensor(0.0032, grad_fn=<CopyBackwards>) tensor(0.0008, grad_fn=<DivBackward0>) tensor(0.0002, grad_fn=<MaxBackward1>)
activ quant

In [63]:
model_scaled.quant_feat.__dict__


{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 'num_bits': 30,
 'enforce_true_quantized': True,
 'delta_m': None,
 'max_absolute': 8.0,
 'stat_max_absolute': 1e-10,
 'pre_scaling_factor': 0.25}

In [51]:
from extra_layers import _Quantize_stats
results = {}
for qb in [None,12]:
    config_scaled.model['num_bits_feat'] = qb #10 #12
    model_scaled = make_model(**config_scaled.model).to(device)
    if qb is None:   # No rescaling of h with qb=None
        new_weights['recurrent_layer.weight'] = float_recurrent_weight
        #model_scaled.load_state_dict(new_weights_normalphaU)
    else:
        new_weights['recurrent_layer.weight'] = requant_recurrent_weight
    model_scaled.load_state_dict(new_weights)
    if qb is not None:      
        model_scaled.quant_input.pre_scaling_factor = input_pre_scaling_factor     
        model_scaled.quant_input.max_absolute = saved_quant_input_max_absolute #saved_quant_input_max_absolute  ## set it twice to get 1 as max
        model_scaled.quant_feat.max_absolute = new_quant_feat_max_absolute       
        model_scaled.quant_feat.pre_scaling_factor = saved_alpha_w     
        #print(model_scaled.activation.b)
        #print(quant_bias(model_scaled.activation.b))
        if ('activation.b' in new_weights) or ('recurrent_layer.bias' in new_weights):
            #quantize biases
            quant_bias = _Quantize_stats(weight=None,num_bits= model_scaled.quant_feat.num_bits, max_absolute= new_bias_max_absolute)
            if 'activation.b' in new_weights:
                model_scaled.activation.b.data = quant_bias(model_scaled.activation.b) 
            if 'recurrent_layer.bias' in new_weights:
                model_scaled.recurrent_layer.bias.data = quant_bias(model_scaled.recurrent_layer.bias)
    stat_test = evaluate(test_ds, test_batch_size, model_scaled, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device)
    print(qb, stat_test)
    results[qb] =stat_test.copy()

for q in results:
    print(q,results[q])
#save perf in txt file
with open("results/"+expe_name+"_"+sweep_name+"_"+short2fullname[expe_name]+"_quant.txt", "w") as f:
    for q in results:
        for kk in results[q]:
            f.write(str(q)+";"+str(kk)+";"+str(results[q][kk])+"\n")



None {'test_loss': 9.902935005356994e-08}
12 {'test_loss': 2.426349965389818e-05}
None {'test_loss': 9.902935005356994e-08}
12 {'test_loss': 2.426349965389818e-05}


## Verification of the intermediate computation: chek for quantization

need to simulate the QRNN computation and save intermediate values 

In [52]:
model_scaled.quant_feat.num_bits

12

In [53]:
x,y = next(iter(test_ds))
x0 = x[None,:] #No batch_size [0:1]
x0.shape

torch.Size([1, 1000, 10])

In [54]:
if 'activation.b' in new_weights:
    model.activation.b/ saved_alpha_u

In [55]:

if 'activation.b' in new_weights:
    model_scaled.activation.b

In [56]:
x0.shape

torch.Size([1, 1000, 10])

In [57]:
saved_quant_input_max_absolute
input_pre_scaling_factor

1.0

In [58]:

import torch.nn.functional as F
hidden = []
hidden_scaled = []
hidden_scaled_input = []
hidden_scaled_quant = []
hidden_scaled_rec = []
for ii in range(1000):
    print("input",x0[0:1,ii])
    print("qinput",model_scaled.quant_input(x0[0:1,ii])/input_pre_scaling_factor)
    #print(model.input_layer(x0[0:1,ii]))
    #print(model_scaled.input_layer(x0[0:1,ii])*saved_quant_input_max_absolute*saved_alpha_u)
    print("diff U.x",torch.norm(model.input_layer(x0[0:1,ii])-model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))*saved_alpha_u/input_pre_scaling_factor))
    
    if len(hidden)==0:
        if 'recurrent_layer.bias' in new_weights:
                hidden.append(model.input_layer(x0[0:1,ii])+model.recurrent_layer.bias)
                hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))+model_scaled.recurrent_layer.bias)
                hidden_scaled_rec.append(torch.zeros_like(hidden_scaled[0])+model_scaled.recurrent_layer.bias) # fake
        else:
            hidden.append(model.input_layer(x0[0:1,ii]))
            hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii])))
            hidden_scaled_rec.append(torch.zeros_like(hidden_scaled[0])) # fake
        #print(hidden_scaled[0])
        #print(hidden_scaled[0]*2**(config.model['num_bits']))
        hidden_scaled_quant.append(torch.zeros_like(hidden_scaled[0])) # fake
    else:
        #print("hidden",hidden[-1][0,0:10])
        #print("hidden_rec",model.recurrent_layer(hidden[-1])[0,0:10])
        hidden.append(model.input_layer(x0[0:1,ii])+model.recurrent_layer(hidden[-1]))
        #print("hidden_scaled",hidden_scaled[-1][0,0:10])
        #print("hidden_scaled",hidden_scaled[-1][0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        hidden_scaled_quant.append(model_scaled.quant_feat(hidden_scaled[-1]))
        #print("hidden_scaled_quant",hidden_scaled_quant[-1][0,0:10])
        #print("hidden_scaled_quant",hidden_scaled_quant[-1][0,0:10]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor))
        print("diff_hidden_scaled",torch.norm(hidden[-2]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor))
        print("diff_hidden_quant",torch.norm(hidden[-2]-hidden_scaled_quant[-1]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor)))
        diff_hidden_scaled = (hidden_scaled_quant[-1]/model_scaled.quant_feat.pre_scaling_factor-hidden_scaled[-1])*saved_alpha_u/input_pre_scaling_factor
        print("quant diff hidden",torch.norm(diff_hidden_scaled).detach().numpy(),torch.min(torch.abs(diff_hidden_scaled)).detach().numpy(),torch.mean(torch.abs(diff_hidden_scaled)).detach().numpy(),torch.max(torch.abs(diff_hidden_scaled)).detach().numpy(),diff_hidden_scaled.shape)
        #print("hidden_scaled_reccccc",model.recurrent_layer(hidden_scaled_quant[-1]*saved_alpha_u/(input_pre_scaling_factor*model_scaled.quant_feat.pre_scaling_factor))[0,0:10])
        #print("hidden_scaled_reccccc1",model.recurrent_layer(hidden_scaled_quant[-1]/model_scaled.quant_feat.pre_scaling_factor)[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        #print("hidden_scaled_rec",model_scaled.recurrent_layer(hidden_scaled_quant[-1])[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        #print("hidden_scaled_rec",model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))[0,0:10]*saved_alpha_u/input_pre_scaling_factor)
        hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))+model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1])))
        hidden_scaled_rec.append(model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))) 
    
    hidden_scaled_input.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii])))
    #print("hidden_scaled_rec",hidden_scaled_rec[-1]*saved_alpha_u/input_pre_scaling_factor)
    print("diff h_t",torch.norm(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor))
    print("h_t, min,mean,max,norm",hidden[-1].min().detach().numpy(),hidden[-1].mean().detach().numpy(),hidden[-1].max().detach().numpy(),torch.norm(hidden[-1]).detach().numpy())
    #print("diff h_t",hidden[-1][0,0:10])
    #print("diff h_t",hidden_scaled[-1][0,0:10])
    #print("diff h_t",hidden_scaled[-1][0,0:10]*saved_alpha_u/input_pre_scaling_factor)
    #if ii>2:
    #    aaa
    if 'activation.b' in new_weights:
        print("diff h_t+bias",torch.norm(hidden[-1]+model.activation.b-(hidden_scaled[-1]+model_scaled.activation.b)*saved_alpha_u/input_pre_scaling_factor))
    #NO ACTIV IN SSM hidden[-1] = model.activation(hidden[-1])
    #NO ACTIV IN SSM hidden_scaled[-1] = model_scaled.activation(hidden_scaled[-1])
    print("activ",torch.norm(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor),torch.norm(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor)/torch.norm(hidden[-1]),torch.max(torch.abs(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor)))
    #print("activ value",hidden[-1],hidden_scaled[-1]*saved_alpha_u/input_pre_scaling_factor)
    #print("activ valueq",hidden_scaled[-1],model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor)
    print("activ quantif",torch.norm(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.pre_scaling_factor*saved_alpha_u/input_pre_scaling_factor))
    

input tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
qinput tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])
diff U.x tensor(0., grad_fn=<CopyBackwards>)
diff h_t tensor(0.0179, grad_fn=<CopyBackwards>)
h_t, min,mean,max,norm -3.2515633 -0.0084897615 3.7618876 16.906162
activ tensor(0.0179, grad_fn=<CopyBackwards>) tensor(0.0011, grad_fn=<DivBackward0>) tensor(0.0014, grad_fn=<MaxBackward1>)
activ quantif tensor(0.5847, grad_fn=<CopyBackwards>)
input tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
qinput tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
diff U.x tensor(0., grad_fn=<CopyBackwards>)
diff_hidden_scaled tensor(0.0179, grad_fn=<CopyBackwards>)
diff_hidden_quant tensor(0.5847, grad_fn=<CopyBackwards>)
quant diff hidden 0.5847806 0.0 0.022479804 0.043299917 torch.Size([1, 512])
diff h_t tensor(0.5846, grad_fn=<CopyBackwards>)
h_t, min,mean,max,norm -4.996379 -0.015067793 5.690657 25.236826
activ tensor(0.5846, grad_fn=<CopyBackwards>) tensor(0.0232, grad_fn=<DivBack

diff U.x tensor(0., grad_fn=<CopyBackwards>)
diff_hidden_scaled tensor(3.3284, grad_fn=<CopyBackwards>)
diff_hidden_quant tensor(3.4459, grad_fn=<CopyBackwards>)
quant diff hidden 0.5627233 0.0 0.021316966 0.043299917 torch.Size([1, 512])
diff h_t tensor(3.4472, grad_fn=<CopyBackwards>)
h_t, min,mean,max,norm -13.112688 -0.19063479 8.586685 58.59805
activ tensor(3.4472, grad_fn=<CopyBackwards>) tensor(0.0588, grad_fn=<DivBackward0>) tensor(0.5233, grad_fn=<MaxBackward1>)
activ quantif tensor(3.4973, grad_fn=<CopyBackwards>)
input tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
qinput tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
diff U.x tensor(0., grad_fn=<CopyBackwards>)
diff_hidden_scaled tensor(3.4472, grad_fn=<CopyBackwards>)
diff_hidden_quant tensor(3.4973, grad_fn=<CopyBackwards>)
quant diff hidden 0.58085996 0.0 0.022374092 0.043299917 torch.Size([1, 512])
diff h_t tensor(3.4975, grad_fn=<CopyBackwards>)
h_t, min,mean,max,norm -11.427676 -0.023908999 10.495446 59.471657

In [59]:
print(model_scaled)

QSSM(
  (activation): ReLU()
  (input_layer): QLinear(in_features=10, out_features=512, bias=False)
  (recurrent_layer): QLinear(in_features=512, out_features=512, bias=True)
  (output_layer): Linear(in_features=512, out_features=9, bias=True)
  (quant_input): _Quantize_stats()
  (quant_feat): _Quantize_stats()
)


In [60]:

preds = []
if config.model['manytomany']:
    for ii,hh in enumerate(hidden_scaled): #zip(hidden_scaled):
        #print(hh.shape)
        #print(hh[:,0:10])
        preds.append(model_scaled.output_layer(model_scaled.activation(hh)))
    nsmples = 10
else:
    preds.append(model_scaled.output_layer(model_scaled.activation(hidden_scaled[-1])))
    nsmples = 1
    '''if ii==2:
        break'''
    #preds.append(model_scaled.output_layer(hh[0]))
ypred = model_scaled(x0.to(device))
preds = torch.concat(preds,0)
print(ypred[-1:])    
print("preds",preds[-nsmples:])
print(torch.argmax(ypred[-nsmples:],dim=-1))    
print("preds",torch.argmax(preds[-nsmples:],dim=-1))
print(ypred.shape,preds.shape)

tensor([[ 66.1710,  90.4001,  83.2699, 118.5345,  96.8048,  83.4471,  87.4061,
          90.6219,  92.2442]], grad_fn=<SliceBackward0>)
preds tensor([[ 60.6146, 121.7774,  83.6743,  91.0054,  93.0944,  87.9002,  88.1035,
          91.6217,  88.7217],
        [ 62.4902,  92.1335,  90.6568,  96.9405,  85.4834, 121.6068,  93.6237,
          86.7393,  94.0797],
        [ 77.5797,  81.6285, 102.6577,  82.3822,  82.1084,  82.4381,  80.7235,
          82.3260,  80.8269],
        [ 78.1954,  82.8082,  71.7577,  85.6764, 111.2355,  74.9758,  83.6806,
          76.1679,  81.5955],
        [ 64.0168,  96.0230,  94.8161,  87.6663,  99.3767,  88.2571, 125.3367,
          97.8592,  90.0806],
        [ 72.9442,  93.6363,  90.9010,  95.0775,  85.7783,  93.8768,  93.3043,
         112.5697,  87.6182],
        [ 58.2644, 100.9028, 117.8342,  92.7372,  94.5905,  94.7654,  98.1885,
          93.7455,  91.8851],
        [ 72.1435, 121.2415,  84.1608,  80.8647,  86.8823,  81.7435,  87.8093,
          82.182

In [70]:
vvin = verify_and_round(model_scaled.recurrent_layer.weight, config.model['num_bits'])

for hh, hhin, hhq, hhrec in zip(hidden_scaled,hidden_scaled_input, hidden_scaled_quant,hidden_scaled_rec):
    vvin = hhin*(2**(config.model['num_bits']-1+shift_input)) #+model_scaled.quant_input.num_bits-1)) 
    print("hhin ",torch.norm(vvin.round() - vvin), torch.min(vvin.round() - vvin), torch.max(vvin.round() - vvin))
    vvin = verify_and_round(hhin, config.model['num_bits']-1+shift_input)
    vvq = hhq*(2**(model_scaled.quant_input.num_bits-1)) 
    #vvq = hhq*(2**(shift_input+model_scaled.quant_input.num_bits-1)*model_scaled.quant_input.max_absolute) 
    print("hhq ",torch.norm(vvq.round() - vvq), torch.min(vvq.round() - vvq), torch.max(vvq.round() - vvq))
    vvq = verify_and_round(hhq, model_scaled.quant_feat.num_bits-1)  # +1 for multiply by 2 in model_scaled.quant_input.max_absolute
    vvrec = hhrec*(2**(config.model['num_bits']-1+model_scaled.quant_feat.num_bits-1)) 
    print("hhrec ",torch.norm(vvrec.round() - vvrec), torch.min(vvrec.round() - vvrec), torch.max(vvrec.round() - vvrec))
    vvrec = verify_and_round(hhrec, config.model['num_bits']-1+model_scaled.quant_feat.num_bits-1)
    vv = hh*(2**(config.model['num_bits']-1+model_scaled.quant_feat.num_bits-1)) 
    print("hh ",torch.norm(vv.round() - vv), torch.min(vv.round() - vv), torch.max(vv.round() - vv))
    vv = verify_and_round(hh, config.model['num_bits']-1+model_scaled.quant_feat.num_bits-1)
    
    

1 max 2
hhin  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
1 max 1
hhq  tensor(0.) tensor(0.) tensor(0.)
1 max 11
hhrec  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
11 max 12
hh  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
11 max 12
hhin  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
1 max 1
hhq  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
8 max 11
hhrec  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
11 max 12
hh  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
11 max 12
hhin  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<Ma

# Old stuf 

In [19]:
results = {}
for qb in [None,6,8,10,12]:
    config.model['num_bits_feat'] = qb #10 #12

    model = make_model(**config.model).to(device)
    model.load_state_dict(torch.load(weight_file_path))
    if qb is not None:
        model.quant_input.max_absolute = saved_quant_input_max_absolute  ## set it twice to get 1 as max
        model.quant_feat.max_absolute = saved_quant_feat_max_absolute       
    stat_test = evaluate(test_ds, test_batch_size, model, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device)
    print(qb, stat_test)
    results[qb] =stat_test.copy()

for q in results:
    print(q,results[q])

use bjorck1
ModuleDict(
  (weight): ParametrizationList(
    (0): _SpectralNorm()
    (1): _BjorckNorm()
    (2): _Quantize()
  )
)
True
bjorckbbbbbbbbbbbbbbbbbbbbbbbbbbb
None {'test_loss': 9.237733320333064e-05}
use bjorck1
ModuleDict(
  (weight): ParametrizationList(
    (0): _SpectralNorm()
    (1): _BjorckNorm()
    (2): _Quantize()
  )
)
True
bjorckbbbbbbbbbbbbbbbbbbbbbbbbbbb
6 {'test_loss': 0.9153663516044617}
use bjorck1
ModuleDict(
  (weight): ParametrizationList(
    (0): _SpectralNorm()
    (1): _BjorckNorm()
    (2): _Quantize()
  )
)
True
bjorckbbbbbbbbbbbbbbbbbbbbbbbbbbb
8 {'test_loss': 0.02758418396115303}
use bjorck1
ModuleDict(
  (weight): ParametrizationList(
    (0): _SpectralNorm()
    (1): _BjorckNorm()
    (2): _Quantize()
  )
)
True
bjorckbbbbbbbbbbbbbbbbbbbbbbbbbbb
10 {'test_loss': 0.00037850963417440653}
use bjorck1
ModuleDict(
  (weight): ParametrizationList(
    (0): _SpectralNorm()
    (1): _BjorckNorm()
    (2): _Quantize()
  )
)
True
bjorckbbbbbbbbbbbbbbbbb

In [41]:
import copy
config_scaled = copy.deepcopy(config)

In [42]:
config_scaled.model['num_bits']  = None
config_scaled.model['num_bits_feat'] = None
config_scaled.model['name'] = 'QRNN'
config_scaled.model['bias'] = False

In [43]:
config_scaled.model

{'activation': modrelu(),
 'activation_config': {'features': 256},
 'activation_final': None,
 'activation_final_config': None,
 'bias': False,
 'bias_final': True,
 'hidden_size': 256,
 'input_size': 10,
 'manytomany': True,
 'name': 'QRNN',
 'num_bits': None,
 'output_size': 9,
 'use_bjorck': True,
 'reset_type': 'copyb',
 'num_bits_feat': None}

In [44]:

model_scaled = make_model(**config_scaled.model).to(device)
weights = torch.load(weight_file_path)


In [47]:
new_weights = {}
new_weights["input_layer.weight"] = 2**(config.model['num_bits']-1)*model.input_layer.qweight/ saved_alpha_u #NO_PARAM model.input_layer.weight
new_weights['recurrent_layer.weight'] = model.recurrent_layer.weight
new_weights['activation.b'] = 2**(config.model['num_bits']-1)*weights['activation.b']/ saved_alpha_u
new_weights['output_layer.weight'] = saved_alpha_u*weights['output_layer.weight']/2**(config.model['num_bits']-1)

for kk in weights.keys():
    if (kk not in new_weights) and ('input_layer' not in kk) and ('recurrent_layer' not in kk):
        new_weights[kk] = weights[kk]

In [48]:
new_weights.keys()

dict_keys(['input_layer.weight', 'recurrent_layer.weight', 'activation.b', 'output_layer.weight', 'output_layer.bias'])

weights['input_layer.parametrizations.weight.original'] = 2**(config.model['num_bits']-1)*model.input_layer.weight/ saved_alpha_u
weights['activation.b'] = 2**(config.model['num_bits']-1)*weights['activation.b']/ saved_alpha_u
print(weights['input_layer.parametrizations.weight.original'].unique())
weights['output_layer.weight'] = saved_alpha_u*weights['output_layer.weight']/2**(config.model['num_bits']-1)

In [49]:
model_scaled

QRNN(
  (activation): modrelu()
  (input_layer): QLinear(in_features=10, out_features=256, bias=False)
  (recurrent_layer): QLinear(in_features=256, out_features=256, bias=False)
  (output_layer): Linear(in_features=256, out_features=9, bias=True)
)

In [50]:
model_scaled.load_state_dict(new_weights)

<All keys matched successfully>

In [51]:
x,y = next(iter(test_ds))

In [35]:
preds = model(x[0:1])

In [36]:
preds_scaled = model_scaled(x[0:1])

In [37]:
preds_scaled[0,1,:]

tensor([ 6.6481, -2.6134, -2.7067, -2.9618, -2.5944, -3.0028, -2.7067, -3.3808,
        -2.5686], grad_fn=<SliceBackward0>)

In [51]:
torch.norm(torch.abs(preds[0,:500,:]-preds_scaled[0,:500,:]))

tensor(0.0199, grad_fn=<CopyBackwards>)

In [53]:
stat_test = evaluate(test_ds, test_batch_size, model_scaled, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device)
print(stat_test)

{'test_loss': 9.224341920344159e-05}


In [55]:
x0 = x[0:1]

In [62]:
hidden = []
hidden_scaled = []
for ii in range(3):
    print(torch.norm(model.input_layer(x0[0:1,ii])-model_scaled.input_layer(x0[0:1,ii])*saved_alpha_u/2**(config.model['num_bits']-1)))
    if len(hidden)==0:
        hidden.append(model.input_layer(x0[0:1,ii]))
        hidden_scaled.append(model_scaled.input_layer(x0[0:1,ii]))
    else:
        hidden.append(model.input_layer(x0[0:1,ii])+model.recurrent_layer(hidden[-1]))
        hidden_scaled.append(model_scaled.input_layer(x0[0:1,ii])+model_scaled.recurrent_layer(hidden_scaled[-1]))
    print(torch.norm(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/2**(config.model['num_bits']-1)))
    hidden[-1] = model.activation(hidden[-1])
    hidden_scaled[-1] = model_scaled.activation(hidden_scaled[-1])
    print(torch.norm(hidden[-1]-hidden_scaled[-1]*saved_alpha_u/2**(config.model['num_bits']-1)))
    

tensor(0., grad_fn=<CopyBackwards>)
tensor(0., grad_fn=<CopyBackwards>)
tensor(1.5130e-05, grad_fn=<CopyBackwards>)
tensor(0., grad_fn=<CopyBackwards>)
tensor(1.5149e-05, grad_fn=<CopyBackwards>)
tensor(2.7136e-05, grad_fn=<CopyBackwards>)
tensor(0., grad_fn=<CopyBackwards>)
tensor(2.7172e-05, grad_fn=<CopyBackwards>)
tensor(3.9127e-05, grad_fn=<CopyBackwards>)


In [54]:
new_saved_quant_feat_max_absolute =  2048 #2**(config.model['num_bits']-1)*saved_quant_feat_max_absolute/saved_alpha_u
#1042.9763707232705

In [55]:
print(new_saved_quant_feat_max_absolute)

2048


## Expe with nor by 2.alpha_U only

In [19]:
def round_values(vv, num_bits):
    vv = vv*2**num_bits
    assert torch.max(torch.abs(vv.round() - vv))<1e-5
    vv = vv.round()
    vv = vv/2**num_bits
    return vv

def verify_round_max(vv, num_bits):
    eval_numbits = 0
    for ss in range(num_bits):
        vv = vv*2
        if torch.max(torch.abs(vv.round() - vv))<1e-5:
            eval_numbits = ss+1
            break
    assert eval_numbits > 0
    print(eval_numbits,"max",num_bits)
    return eval_numbits
    
  
def verify_and_round(vv, num_bits):  
    eval_numbits = verify_round_max(vv, num_bits)
    return round_values(vv, eval_numbits)


In [20]:
weights = torch.load(weight_file_path)

In [21]:
new_weights_normalphaU = {}
requant_input_weight = round_values(model.input_layer.qweight/ (2*saved_alpha_u), config.model['num_bits']) #NO_PARAM model.input_layer.weight
new_weights_normalphaU["input_layer.weight"] = requant_input_weight #model.input_layer.weight/ (2*saved_alpha_u)  
#new_weights_normalphaU['recurrent_layer.weight'] = model.recurrent_layer.weight / saved_alpha_w ## compensate alpha_w in hidden quantization
#requant_recurrent_weight = round_values(model.recurrent_layer.weight, config.model['num_bits'])
new_weights_normalphaU['recurrent_layer.weight'] = model.recurrent_layer.weight  
scaled_quant_feat_max_absolute  = saved_quant_feat_max_absolute/(2*saved_alpha_u)
print(scaled_quant_feat_max_absolute) #16.296505792551102
new_quant_feat_max_absolute  = 16 #saved_alpha_w*saved_quant_feat_max_absolute/(2*saved_alpha_u)
#new_quant_feat_max_absolute  = scaled_quant_feat_max_absolute #8/saved_alpha_w #saved_alpha_w*saved_quant_feat_max_absolute/(2*saved_alpha_u)
new_weights_normalphaU['activation.b'] = weights['activation.b']/ (2*saved_alpha_u)
new_weights_normalphaU['output_layer.weight'] = 2*saved_alpha_u*weights['output_layer.weight']

for kk in weights.keys():
    if (kk not in new_weights_normalphaU) and ('input_layer' not in kk) and ('recurrent_layer' not in kk):
        new_weights_normalphaU[kk] = weights[kk]

16.296505792551102


### verif is U and W are in Q_{nbits-1} (+ factor 2 for U) 

In [22]:
iww = 2**(config.model['num_bits']-1)*2*new_weights_normalphaU["input_layer.weight"].unique()
print("input ",torch.norm(iww.round() - iww))

iww = 2**(config.model['num_bits']-1)*new_weights_normalphaU['recurrent_layer.weight']/saved_alpha_w
#print(iww)
print("recurrent ",torch.norm(iww.round() - iww))

input  tensor(0., grad_fn=<CopyBackwards>)
recurrent  tensor(8.7021e-05, grad_fn=<CopyBackwards>)


In [23]:
import copy
config_scaled = copy.deepcopy(config)

config_scaled.model['num_bits']  = None
config_scaled.model['num_bits_feat'] = None
config_scaled.model['name'] = 'QRNN'
config_scaled.model['bias'] = False

In [24]:
results = {}
for qb in [None,6,8,10,12]:
    config_scaled.model['num_bits_feat'] = qb #10 #12

    model_scaled = make_model(**config_scaled.model).to(device)
    model_scaled.load_state_dict(new_weights_normalphaU)
    if qb is not None:
        model_scaled.quant_input.max_absolute = saved_quant_input_max_absolute  ## set it twice to get 1 as max
        model_scaled.quant_feat.max_absolute = new_quant_feat_max_absolute       
        #model_scaled.quant_feat.pre_scaling_factor = saved_alpha_w       
    stat_test = evaluate(test_ds, test_batch_size, model_scaled, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device)
    print(qb, stat_test)
    results[qb] = stat_test.copy()

for q in results:
    print(q,results[q])

None {'test_loss': 9.133988351095468e-05}
6 {'test_loss': 0.900307297706604}
8 {'test_loss': 0.024784889072179794}
10 {'test_loss': 0.00043290978646837175}
12 {'test_loss': 0.00010541886877035722}
None {'test_loss': 9.133988351095468e-05}
6 {'test_loss': 0.900307297706604}
8 {'test_loss': 0.024784889072179794}
10 {'test_loss': 0.00043290978646837175}
12 {'test_loss': 0.00010541886877035722}


## Expe with nor by 2.alpha_U , and integration of alpha_W

In [25]:

weights = torch.load(weight_file_path)

In [26]:
new_weights = {}
requant_input_weight = round_values(model.input_layer.qweight/ (2*saved_alpha_u), config.model['num_bits']) #NO_PARAM model.input_layer.weight
new_weights["input_layer.weight"] = requant_input_weight #model.input_layer.weight/ (2*saved_alpha_u)
requant_recurrent_weight = round_values(model.recurrent_layer.weight/ saved_alpha_w, config.model['num_bits'])
new_weights['recurrent_layer.weight'] = requant_recurrent_weight #model.recurrent_layer.weight / saved_alpha_w ## compensate alpha_w in hidden quantization
#new_weights['recurrent_layer.weight'] = model.recurrent_layer.weight  ## compensate alpha_w in hidden quantization
scaled_quant_feat_max_absolute  = saved_quant_feat_max_absolute/(2*saved_alpha_u)
print(scaled_quant_feat_max_absolute) #saved_quant_feat_max_absolute*saved_alpha_w/(2*saved_alpha_u))
print(scaled_quant_feat_max_absolute*saved_alpha_w)  #4.055123978191649
new_quant_feat_max_absolute  = 4 #saved_alpha_w*saved_quant_feat_max_absolute/(2*saved_alpha_u)
#new_quant_feat_max_absolute  = scaled_quant_feat_max_absolute #8/saved_alpha_w #saved_alpha_w*saved_quant_feat_max_absolute/(2*saved_alpha_u)
new_weights['activation.b'] = weights['activation.b']/ (2*saved_alpha_u)
new_weights['output_layer.weight'] = 2*saved_alpha_u*weights['output_layer.weight']

for kk in weights.keys():
    if (kk not in new_weights) and ('input_layer' not in kk) and ('recurrent_layer' not in kk):
        new_weights[kk] = weights[kk]

16.296505792551102
4.055123978191649


### verif is U and W are in Q_{nbits-1} (+ factor 2 for U) 

In [27]:
iww = 2**(config.model['num_bits']-1)*2*new_weights["input_layer.weight"].unique()
print("input ",torch.norm(iww.round() - iww))

iww = 2**(config.model['num_bits']-1)*new_weights['recurrent_layer.weight']
#print(iww)
print("recurrent ",torch.norm(iww.round() - iww))
#print("recurrent ",(iww.round() - iww).unique())
#print(torch.max(model.recurrent_layer.parametrizations['weight'][1](model.recurrent_layer.parametrizations['weight'].original)))
#print(torch.min(model.recurrent_layer.parametrizations['weight'][1](model.recurrent_layer.parametrizations['weight'].original)))
#vvv = 2**(config.model['num_bits']-1)*model.recurrent_layer.weight / saved_alpha_w
#print((vvv.round() - vvv).unique())

input  tensor(0., grad_fn=<CopyBackwards>)
recurrent  tensor(0., grad_fn=<CopyBackwards>)


In [28]:
import copy
config_scaled = copy.deepcopy(config)

config_scaled.model['num_bits']  = None
config_scaled.model['num_bits_feat'] = None
config_scaled.model['name'] = 'QRNN'
config_scaled.model['bias'] = False

In [29]:
results = {}
for qb in [None,6,8,10,12]:
    config_scaled.model['num_bits_feat'] = qb #10 #12
    model_scaled = make_model(**config_scaled.model).to(device)
    if qb is None:   # No rescaling of h with qb=None
        model_scaled.load_state_dict(new_weights_normalphaU)
    else:
        model_scaled.load_state_dict(new_weights)
    if qb is not None:
        model_scaled.quant_input.max_absolute = saved_quant_input_max_absolute  ## set it twice to get 1 as max
        model_scaled.quant_feat.max_absolute = new_quant_feat_max_absolute       
        model_scaled.quant_feat.pre_scaling_factor = saved_alpha_w       
    stat_test = evaluate(test_ds, test_batch_size, model_scaled, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device)
    print(qb, stat_test)
    results[qb] =stat_test.copy()

for q in results:
    print(q,results[q])

None {'test_loss': 9.133988351095468e-05}
6 {'test_loss': 0.8989794850349426}
8 {'test_loss': 0.028380531817674637}
10 {'test_loss': 0.0004319324507378042}
12 {'test_loss': 0.00010572293831501156}
None {'test_loss': 9.133988351095468e-05}
6 {'test_loss': 0.8989794850349426}
8 {'test_loss': 0.028380531817674637}
10 {'test_loss': 0.0004319324507378042}
12 {'test_loss': 0.00010572293831501156}


## Expe with quantized bias + nor by 2.alpha_U , and integration of alpha_W

In [29]:
weights = torch.load(weight_file_path)

In [30]:
new_weights = {}
requant_input_weight = round_values(model.input_layer.qweight/ (2*saved_alpha_u), config.model['num_bits']) #NO_PARAM model.input_layer.weight
new_weights["input_layer.weight"] = requant_input_weight #model.input_layer.weight/ (2*saved_alpha_u)
requant_recurrent_weight = round_values(model.recurrent_layer.weight/ saved_alpha_w, config.model['num_bits'])
new_weights['recurrent_layer.weight'] = requant_recurrent_weight #model.recurrent_layer.weight / saved_alpha_w ## compensate alpha_w in hidden quantization
#new_weights['recurrent_layer.weight'] = model.recurrent_layer.weight  ## compensate alpha_w in hidden quantization
scaled_quant_feat_max_absolute  = saved_quant_feat_max_absolute/(2*saved_alpha_u)
print(scaled_quant_feat_max_absolute) #saved_quant_feat_max_absolute*saved_alpha_w/(2*saved_alpha_u))
print(scaled_quant_feat_max_absolute*saved_alpha_w)  #4.055123978191649
new_quant_feat_max_absolute  = 4 #saved_alpha_w*saved_quant_feat_max_absolute/(2*saved_alpha_u)
#new_quant_feat_max_absolute  = scaled_quant_feat_max_absolute #8/saved_alpha_w #saved_alpha_w*saved_quant_feat_max_absolute/(2*saved_alpha_u)
new_weights['activation.b'] = weights['activation.b']/ (2*saved_alpha_u)
new_weights['output_layer.weight'] = 2*saved_alpha_u*weights['output_layer.weight']

for kk in weights.keys():
    if (kk not in new_weights) and ('input_layer' not in kk) and ('recurrent_layer' not in kk):
        new_weights[kk] = weights[kk]

16.296505792551102
4.055123978191649


In [33]:
import copy
config_scaled = copy.deepcopy(config)

config_scaled.model['num_bits']  = None
config_scaled.model['num_bits_feat'] = None
config_scaled.model['name'] = 'QRNN'
config_scaled.model['bias'] = False
config_scaled.model['activation'] = layers.modified4q_modrelu(**config_scaled.model['activation_config'])

modified modReLU


In [34]:
print(config_scaled.model)

{'activation': modified4q_modrelu(), 'activation_config': {'features': 256}, 'activation_final': None, 'activation_final_config': None, 'bias': False, 'bias_final': True, 'hidden_size': 256, 'input_size': 10, 'manytomany': True, 'name': 'QRNN', 'num_bits': None, 'output_size': 9, 'use_bjorck': True, 'reset_type': 'copyb', 'num_bits_feat': None}


In [35]:
from quantized_layers import _Quantize_stats
results = {}
for qb in [None,6,8,10,12]:
    config_scaled.model['num_bits_feat'] = qb #10 #12
    model_scaled = make_model(**config_scaled.model).to(device)
    if qb is None:   # No rescaling of h with qb=None
        model_scaled.load_state_dict(new_weights_normalphaU)
    else:
        model_scaled.load_state_dict(new_weights)
    if qb is not None:
        model_scaled.quant_input.max_absolute = saved_quant_input_max_absolute  ## set it twice to get 1 as max
        model_scaled.quant_feat.max_absolute = new_quant_feat_max_absolute       
        model_scaled.quant_feat.pre_scaling_factor = saved_alpha_w     
        #quantize biases
        quant_bias = _Quantize_stats(weight=None,num_bits= model_scaled.quant_feat.num_bits, max_absolute= model_scaled.quant_feat.max_absolute)
        #print(model_scaled.activation.b)
        #print(quant_bias(model_scaled.activation.b))
        model_scaled.activation.b.data = quant_bias(model_scaled.activation.b) 
    stat_test = evaluate(test_ds, test_batch_size, model_scaled, loss_fn=config.train['loss_fn'], metrics=config.train['metrics'], kind='test', torch_device=device)
    print(qb, stat_test)
    results[qb] =stat_test.copy()

for q in results:
    print(q,results[q])

None {'test_loss': 9.226499969372526e-05}
6 {'test_loss': 0.7314143776893616}
8 {'test_loss': 0.030396342277526855}
10 {'test_loss': 0.00039048207690939307}
12 {'test_loss': 0.00010659168765414506}
None {'test_loss': 9.226499969372526e-05}
6 {'test_loss': 0.7314143776893616}
8 {'test_loss': 0.030396342277526855}
10 {'test_loss': 0.00039048207690939307}
12 {'test_loss': 0.00010659168765414506}


In [36]:
x,y = next(iter(test_ds))
x0 = x[0:1]

In [73]:
config_scaled.model

{'activation': ReLU(),
 'activation_config': None,
 'activation_final': None,
 'activation_final_config': None,
 'basic_block': 4,
 'bias': True,
 'bias_final': True,
 'givens': 2,
 'hidden_size': 512,
 'input_size': 10,
 'manytomany': True,
 'multiparameters': False,
 'name': 'QSSM',
 'num_bits': None,
 'output_size': 9,
 'seed': 171,
 'single_layer': True,
 'num_bits_feat': 12}

In [37]:
print(model_scaled.quant_feat.__dict__)

{'training': False, '_parameters': OrderedDict(), '_buffers': OrderedDict(), '_non_persistent_buffers_set': set(), '_backward_hooks': OrderedDict(), '_is_full_backward_hook': None, '_forward_hooks': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_state_dict_hooks': OrderedDict(), '_load_state_dict_pre_hooks': OrderedDict(), '_modules': OrderedDict(), 'num_bits': 12, 'enforce_true_quantized': True, 'delta_m': None, 'max_absolute': 4, 'stat_max_absolute': 1e-10, 'pre_scaling_factor': 0.24883395433425903}


In [39]:
print(torch.norm(model.recurrent_layer.weight-model_scaled.recurrent_layer.weight*saved_alpha_w))

tensor(0., grad_fn=<CopyBackwards>)


In [40]:
hidden = []
hidden_scaled = []
hidden_scaled_input = []
hidden_scaled_quant = []
hidden_scaled_rec = []
for ii in range(10):
    print(x0[0:1,ii])
    print(model_scaled.quant_input(x0[0:1,ii]))
    #print(model.input_layer(x0[0:1,ii]))
    #print(model_scaled.input_layer(x0[0:1,ii])*2*saved_alpha_u)
    print(torch.norm(model.input_layer(x0[0:1,ii])-model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))*2*saved_alpha_u))
    
    if len(hidden)==0:
        hidden.append(model.input_layer(x0[0:1,ii]))
        hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii])))
        #print(hidden_scaled[0])
        #print(hidden_scaled[0]*2**(config.model['num_bits']))
        hidden_scaled_quant.append(torch.zeros_like(hidden_scaled[0])) # fake
        hidden_scaled_rec.append(torch.zeros_like(hidden_scaled[0])) # fake
    else:
        #print(model.recurrent_layer(hidden[-1]))
        #print(model_scaled.recurrent_layer(hidden_scaled[-1])*2*saved_alpha_u/4.0)
        #print(hidden_scaled[-1])
        #print(model_scaled.quant_feat(hidden_scaled[-1])/saved_alpha_w)
        '''print("h -sh",torch.norm(hidden[-1]-hidden_scaled[-1]*2*saved_alpha_u))
        print(hidden[-1][0,0:10])
        print(hidden_scaled[-1][0,0:10]*2*saved_alpha_u)
        print(model_scaled.quant_feat(hidden_scaled[-1])[0,0:10]*2*saved_alpha_u/saved_alpha_w)A
        print(2**(model_scaled.quant_feat.num_bits-1))
        print(model_scaled.quant_feat.max_absolute)
        print(hidden_scaled[-1])
        #hidden_scaled[-1][0,1] = -8
        print(2**(model_scaled.quant_feat.num_bits-1)*model_scaled.quant_feat(hidden_scaled[-1])/model_scaled.quant_feat.max_absolute)
        print(torch.abs(hidden[-1]-hidden_scaled[-1]*2*saved_alpha_u)[0,0:10])
        print(torch.abs(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w)[0,0:10])
        print(torch.abs(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w)[0,0:10]-torch.abs(hidden[-1]-hidden_scaled[-1]*2*saved_alpha_u)[0,0:10])
        print((torch.square(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w)-torch.square(hidden[-1]-hidden_scaled[-1]*2*saved_alpha_u)).min())
        print((torch.square(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w)-torch.square(hidden[-1]-hidden_scaled[-1]*2*saved_alpha_u)).mean())
        print((torch.square(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w)-torch.square(hidden[-1]-hidden_scaled[-1]*2*saved_alpha_u)).max())

        print(torch.sum(torch.square(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w))-torch.sum(torch.square(hidden[-1]-hidden_scaled[-1]*2*saved_alpha_u)))
        #print(torch.norm(hidden_scaled[-1]-model_scaled.quant_feat(hidden_scaled[-1])/saved_alpha_w))
        print(torch.sum(torch.square(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w)))
        print(torch.norm(hidden[-1]-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w))
        print(torch.sum(torch.square(hidden_scaled[-1]*2*saved_alpha_u-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w)))
        print("aaa",torch.norm(hidden_scaled[-1]*2*saved_alpha_u-model_scaled.quant_feat(hidden_scaled[-1])*2*saved_alpha_u/saved_alpha_w))
        print(torch.norm(model.recurrent_layer(hidden[-1])-model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))*2*saved_alpha_u))'''
        #print(model.recurrent_layer(hidden[-1]))
        #print(model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))*2*saved_alpha_u)
        hidden.append(model.input_layer(x0[0:1,ii])+model.recurrent_layer(hidden[-1]))
        hidden_scaled.append(model_scaled.input_layer(model_scaled.quant_input(x0[0:1,ii]))+model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1])))
        hidden_scaled_quant.append(model_scaled.quant_feat(hidden_scaled[-1]))
        hidden_scaled_rec.append(model_scaled.recurrent_layer(model_scaled.quant_feat(hidden_scaled[-1]))) 
    
    hidden_scaled_input.append(model_scaled.input_layer(model_scaled.quant_input(model_scaled.quant_input(x0[0:1,ii]))))
    print(torch.norm(hidden[-1]-hidden_scaled[-1]*2*saved_alpha_u))
    hidden[-1] = model.activation(hidden[-1])
    hidden_scaled[-1] = model_scaled.activation(hidden_scaled[-1])
    print("activ",torch.norm(hidden[-1]-hidden_scaled[-1]*2*saved_alpha_u))
    

tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])
tensor(0., grad_fn=<CopyBackwards>)
tensor(0., grad_fn=<CopyBackwards>)
activ tensor(0.0373, grad_fn=<CopyBackwards>)
tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
tensor(0., grad_fn=<CopyBackwards>)
tensor(0.0567, grad_fn=<CopyBackwards>)
activ tensor(0.0588, grad_fn=<CopyBackwards>)
tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
tensor(0., grad_fn=<CopyBackwards>)
tensor(0.0707, grad_fn=<CopyBackwards>)
activ tensor(0.0703, grad_fn=<CopyBackwards>)
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])
tensor(0., grad_fn=<CopyBackwards>)
tensor(0.0859, grad_fn=<CopyBackwards>)
activ tensor(0.0871, grad_fn=<CopyBackwards>)
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])
tensor

In [41]:
print(len(hidden_scaled))

10


In [42]:
torch.max(torch.abs(model_scaled.activation.b))
vvin = verify_and_round(model_scaled.activation.b, model_scaled.quant_feat.num_bits)

9 max 12


In [43]:
vvin = verify_and_round(model_scaled.recurrent_layer.weight, config.model['num_bits'])

for hh, hhin, hhq, hhrec in zip(hidden_scaled,hidden_scaled_input, hidden_scaled_quant,hidden_scaled_rec):
    vvin = hhin*(2**(config.model['num_bits'])) #+model_scaled.quant_input.num_bits-1)) 
    print("hhin ",torch.norm(vvin.round() - vvin), torch.min(vvin.round() - vvin), torch.max(vvin.round() - vvin))
    vvin = verify_and_round(hhin, config.model['num_bits'])
    vvq = hhq*(2**(model_scaled.quant_input.num_bits-1)*model_scaled.quant_input.max_absolute) 
    print("hhq ",torch.norm(vvq.round() - vvq), torch.min(vvq.round() - vvq), torch.max(vvq.round() - vvq))
    vvq = verify_and_round(hhq, model_scaled.quant_input.num_bits-3)
    vvrec = hhrec*(2**(config.model['num_bits']+model_scaled.quant_input.num_bits-1)*model_scaled.quant_input.max_absolute) 
    print("hhrec ",torch.norm(vvrec.round() - vvrec), torch.min(vvrec.round() - vvrec), torch.max(vvrec.round() - vvrec))
    vvrec = verify_and_round(hhrec, config.model['num_bits']+model_scaled.quant_input.num_bits+3)
    '''sgn = torch.sgn(hhin + hhrec)
    toto = nn.functional.relu(torch.abs(hhin + hhrec) +model_scaled.activation.b)
    toto2 = sgn*toto
    print("toto2")
    toto = verify_and_round(toto2, config.model['num_bits']+model_scaled.quant_input.num_bits+3)
    toto = model_scaled.activation(hhin+hhrec) #hhin + hhrec +model_scaled.activation.b
    print(toto2)#torch.abs(hhin + hhrec) +model_scaled.activation.b)
    print(toto)
    print(torch.max(torch.abs(toto-toto2)))
    print("toto")
    vv = toto2
    for ss in range(10):
        vv = vv*2
        print(ss+1,torch.norm(vv.round() - vv), torch.min(vv.round() - vv), torch.max(vv.round() - vv))
        if torch.max(torch.abs(vv.round() - vv))<1e-5:
            print("ok",ss+1)
            break
    toto = verify_and_round(toto, config.model['num_bits']+model_scaled.quant_input.num_bits+3)'''
    vv = hh*(2**(config.model['num_bits']+model_scaled.quant_feat.num_bits-1)) 
    print("hh ",torch.norm(vv.round() - vv), torch.min(vv.round() - vv), torch.max(vv.round() - vv))
    vv = verify_and_round(hh, config.model['num_bits']+model_scaled.quant_input.num_bits+3)
    
    

5 max 6
hhin  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
6 max 6
hhq  tensor(0.) tensor(0.) tensor(0.)
1 max 9
hhrec  tensor(0.) tensor(0.) tensor(0.)
1 max 21
hh  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
9 max 21
hhin  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
6 max 6
hhq  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
9 max 9
hhrec  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
14 max 21
hh  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
14 max 21
hhin  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad_fn=<MinBackward1>) tensor(0., grad_fn=<MaxBackward1>)
6 max 6
hhq  tensor(0., grad_fn=<CopyBackwards>) tensor(0., grad

$U = \alpha_U.U_k$, $U_k\in[[-2^{k-1},2^{k-1}-1]]$

$W = \alpha_W.W_k$, $W_k\in[[-2^{k-1},2^{k-1}-1]]$

$x = x_{ka}/2^{ka-2}$  , $x_{ka}\in{0,2^{ka-2}}$ #$[-2^{ka-1},2^{ka-1}-1]]$

$h = \alpha_h.h_{ka}$, $h_{ka}\in[[-2^{ka-1},2^{ka-1}-1]]$

$h_{t+1} = W.h + U.x = (\alpha_h*\alpha_W)*(W_k.h_{ka}) + (\alpha_U/2^{ka-2})*(U_k*x_{ka})$

$h_{t+1} = ReLU(h_{t+1})$

On peut eviter la multiplaction par $\alpha_U$

Simplification si 

$1/\alpha_h*\alpha_W = 2^k'$


# Reminder fixed point notation  $Q_{l,k}$

$Q_{l,k} = {p/2^{k},p\in[[-2^{l-1},2^{l-1}-1]]}\in [-2^{l-k-1},2^{l-k-1}[$ with $l$ bits

in particular : $Q_k = Q_{k+1,k} = {p/2^{k},p\in[[-2^k,2^k-1]]}\in [-1,1[$ with $k+1$ bits

multiplication $Q_{l,k}.Q_{l',k'} -> Q_{l+l'-1,k+k'}$

$Q_{k}.Q_{k'} -> Q_{k+k'+1,k+k'}=Q_{k+k'}$

Addition (has to be with the same $k$)

$Q_{k} + Q_{k} -> Q_{k+2,k}$




# Formulation with fixed point variables
## matrices and vectors
$U = \alpha_U/2^{k-1}.U_k= \alpha_U.U'_k$, with $U'_k\in Q_{k-1}$

$W = \alpha_W/2^{k-1}.W_k= \alpha_W.W'_k$, with $W'_k\in Q_{k-1}$

$x(t) = 2*x_{ka}(t)$, with $x\in\{0,1\}$ , $x_{ka}\in\{0,0.5\}$ could be $Q_1$ (2 bits), but easier to keep it in $Q_{ka-1}$

$h(t) = \alpha_h/2^{ka-1}.h_{ka}(t)=\alpha_h.h'_{ka}(t)$ with $h'_{ka}(t)\in Q_{ka-1}$

## recurrent computation

$h(t+1) = W.h(t) + U.x(t) = (\alpha_h*\alpha_W)*(W'_k.h'_{ka}(t)) + (2\alpha_U)*(U'_k*x_{ka}(t))$ with $W'_k.h'_{ka}(t)$ and $U'_k*x_{ka}(t)\in Q_{k+ka-2}$

$h(t+1) = ReLU(h(t+1))$

or $h(t+1) = ModReLU(h(t+1),b) = sign(h(t+1))*ReLU(|h(t+1)|+b)$ => Could be quantized back to $\alpha_h.h'_{ka}(t+1)$



# Scaling hidden vector to reduce floating point multiplications ($2\alpha_U$)

To avoid the multiplication by  par $2\alpha_U$

scaled h : $sh(t) = h(t)/(2\alpha_U)$, $sh(t) = \alpha_{sh}.sh'_{ka}(t)$ with $\alpha_{sh} = \alpha_h/(2\alpha_U)$, and $sh'_{ka}(t)=h'_{ka}(t)$

$sh(t+1) = (\alpha_{sh}*\alpha_W)*(W'_k.sh'_{ka}(t)) + U'_k*x_{ka}(t)$  -> Floating point

$sh(t+1) = ReLU(h(t+1))$

$sh(t+1) = ModReLU(h(t+1),sb), with $sb = b/(2\alpha_U)$ => Could be quantized back to $\alpha_{sh}.sh'_{ka}(t+1)$ (division by $\alpha_sh$)

# Removing the  floating point multiplication  $(\alpha_{sh}*\alpha_W)$
set an upper bound for quantization: choose $\alpha_h$ such as 
$(\alpha_{sh}*\alpha_W) = \alpha_h\alpha_W/(2\alpha_U)= 2^{kal}$


 $U'_k*x_{ka}(t)$ in $Q_{k+ka-2} = Q_{k+ka-1,k+ka-2}$

 $W'_k.sh'_{ka}(t)$ in $Q_{k+ka-2} = Q_{k+ka-1,k+ka-2}$

 $2^{kal}W'_k.sh'_{ka}(t)$ in $Q_{k+ka+kal-1,k+ka-2}$

$sh(t+1) \in Q_{k+ka+kal-1,k+ka-2}$ 
ReLU or ModReLU -> **BUT** division by $\alpha_{sh}=2^{kal}/\alpha_W$ before quantizing  to $Q_{ka}$

# Removing the  floating point division  $\alpha_{sh}$ for quantization
require to set  $\alpha_{sh}=2^{kal}$
quantize $\alpha_W \in Q_{kw,kw'}$


 $U'_k*x_{ka}(t)$ in $Q_{k+ka-2} = Q_{k+ka-1,k+ka-2}$

 $W'_k.sh'_{ka}(t)$ in $Q_{k+ka-2} = Q_{k+ka-1,k+ka-2}$

 $\alpha_W.W'_k.sh'_{ka}(t)$ in $Q_{k+ka+kw-2,k+ka+kw'-2}$

$sh(t+1) \in Q_{k+ka+kw-2,k+ka+kw'-2}$ 
ReLU or ModReLU ->  shift by $\alpha_{sh}=2^{kal}$ before quantizing  to $Q_{ka}$